### Spectrogram based Transfer Learning

#### Data cleanup

In [1]:
import matplotlib.pyplot as plt
import os
import subprocess

def run_preprocess(root, length, split):
    for subdir, dirs, files in os.walk(root):
        for directory in dirs:
            subprocess.call(["./preprocess", os.path.join(subdir, directory), length, split])
        break

In [2]:
def load_features(root, split):
    spect_data = []
    spect_label = []
    for subdir, dirs, files in os.walk(root):
        for directory in dirs:
            file_path = os.path.join(subdir, directory, "split", split, "spect")
            for filename in os.listdir(file_path):
                x = plt.imread(os.path.join(file_path, filename))
                spect_data.append(x)
                spect_label.append(directory)
        break
    return spect_data, spect_label

In [3]:
import shutil

def cleanup_split(root):
    for subdir, dirs, files in os.walk(root):
        for directory in dirs:
            shutil.rmtree(os.path.join(subdir, directory, "split"), ignore_errors = True)
        break

In [4]:
def check_data(root, split):
    for subdir, dirs, files in os.walk(root, split):
        for directory in dirs:
            file_path = os.path.join(subdir, directory, "split", split, "spect")
            f_path, f_dirs, f_files = os.walk(file_path).next()
            if len(f_files) != 2:
                print directory
        break

In [5]:
def remove_spects(root, split, count):
    for subdir, dirs, files in os.walk(root, split):
        for directory in dirs:
            file_path = os.path.join(subdir, directory, "split", split, "spect")
            f_path, f_dirs, f_files = os.walk(file_path).next()
            for f in f_files:
                fname = os.path.basename(f)
                if int(os.path.splitext(os.path.splitext(fname)[0])[0]) > int(count):
                    os.remove(os.path.join(subdir, directory, "split", split, "spect", f))
        break

In [6]:
# cleanup_split('audio-train-new')
# run_preprocess('audio-train-new', "540", "12")
remove_spects('audio-train-new', "12", "28")

#### Load spectrograms as matrices

In [7]:
X, y = load_features('audio-train-new', "12")
# cleanup_split('audio-train')

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### One-hot encode the labels

In [9]:
import keras
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

l_enc = LabelEncoder()
l_enc.fit(y_train)
y_train_enc = l_enc.transform(y_train)
y_train_norm = np_utils.to_categorical(y_train_enc)

l_enc.fit(y_test)
y_test_enc = l_enc.transform(y_test)
y_test_norm = np_utils.to_categorical(y_test_enc)

Using TensorFlow backend.


#### Remove unnecessary variables from memory (To clear memory)

In [10]:
del X, y, y_train_enc, y_test_enc, y_train, y_test

#### Build the model

In [11]:
from keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential

model = Sequential()
model.add(Conv2D(8, (3, 3), padding='same',
                 input_shape=(540, 960, 3)))
model.add(Activation('relu'))
model.add(Conv2D(8, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(16, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(16, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Activation('relu'))
model.add(Dense(118))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

#### Train the neural network

In [12]:
epoch = 1
while epoch <= 5:
    model.fit(np.array(X_train), y_train_norm,
              batch_size=32,
              epochs=1,
              verbose=1,
             validation_data=(np.array(X_test[:4000]), y_test_norm[:4000]))
    model.save_weights('spect_model_weights_' + str(epoch) + '.h5')
    epoch += 1

Train on 2737 samples, validate on 685 samples
Epoch 1/1
2737/2737 [==============================] - 2054s 751ms/step - loss: 3.8637 - acc: 0.1564 - val_loss: 2.8428 - val_acc: 0.3387
Train on 2737 samples, validate on 685 samples
Epoch 1/1
2737/2737 [==============================] - 8146s 3s/step - loss: 0.3028 - acc: 0.9415 - val_loss: 2.2928 - val_acc: 0.4161
Train on 2737 samples, validate on 685 samples
Epoch 1/1
2737/2737 [==============================] - 5389s 2s/step - loss: 0.0291 - acc: 0.9923 - val_loss: 1.7713 - val_acc: 0.5270
Train on 2737 samples, validate on 685 samples
Epoch 1/1


KeyboardInterrupt: 

#### Save the model weights to a file

In [ ]:
model.save_weights('spect_model_weights_' + str(epoch) + '.h5')